In [1]:
import pyscf
import ctypes
import numpy as np
from pyscf import dft
from pyscf import gto
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pyscf import lib
from pyscf.dft import libxc
from pyscf.dft import numint
from pyscf.dft import xc_deriv

In [2]:
mol=gto.Mole()
mol.atom='''
    H 0 0 -1
    H 0 0 1
'''#Z-matrix input format
mol.basis='cc-pvdz'
mol.symmetry = True
mol.build()
mypbe = dft.RKS(mol)
mypbe.xc = 'pbe,pbe'
mypbe.kernel()

converged SCF energy = -0.995650542473473


-0.9956505424734725

In [3]:
#default level=3
#default integration grids use Bragg radii for atoms
#Treutler-Ahlrichs radial grids, Becke partitioning for grid weights
grid = dft.gen_grid.Grids(mol)
grid.level = 5  # Higher levels give more accuracy
grid.build()

In [4]:
weights = grids.weights
coords = grids.coords

In [5]:
#numint method
# Evaluate AO values(phi matrix) on this grid
ao_value_grid = numint.eval_ao(mol, coords, deriv=1)
print(ao_value_grid.shape)
print("dimension for phi matrix:",ao_value_grid[0].shape)
#the value for phi matrix and the first derivative
phi=ao_value_grid[0]
phix=ao_value_grid[1]
phiy=ao_value_grid[2]
phiz=ao_value_grid[3]

(4, 20048, 10)
dimension for phi matrix: (20048, 10)


In [6]:
dft.libxc.xc_reference('GGA_X_PBE')

['J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 77, 3865 (1996)',
 'J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 78, 1396 (1997)']

In [7]:
_itrf= lib.load_library('libxc_itrf')
#this function will return Exc and Vxc
def evalexc(rho):
    #create a list for fn_ids and facs(The identifier for the exchange or correlation functional )
    hyb, fn_facs = dft.libxc.parse_xc('GGA_X_PBE,GGA_C_PBE')
    fn_ids = [x[0] for x in fn_facs]
    facs   = [x[1] for x in fn_facs]
    print(fn_facs)
    #The range-separation parameter
    omega=[0] * len(facs)
    #_XC_NVAR = {
    #('HF', 0): (1, 1),
    #('HF', 1): (1, 2),
    #('LDA', 0): (1, 1),
    #('LDA', 1): (1, 2),
    #('GGA', 0): (4, 2),
    #('GGA', 1): (4, 5),
    #('MGGA', 0): (5, 3),
    #('MGGA', 1): (5, 7),
#}
    #only density is the variable
    spin=0
    #nvar: rho, rho_x,rho_y,rho_z
    #xlen: The length may represent the number of independent values related to vxc or energy density computations at each grid point.
    nvar, xlen = xc_deriv._XC_NVAR['GGA', spin]
    ngrids = rho.shape[-1]
    #The order of derivatives to compute 
    deriv=1
    outlen = lib.comb(xlen+deriv, deriv)
    out = np.zeros((outlen,ngrids))
    n = len(fn_ids)
    density_threshold=0
    _itrf.LIBXC_eval_xc(ctypes.c_int(n),
                            (ctypes.c_int*n)(*fn_ids),
                            (ctypes.c_double*n)(*facs),
                            (ctypes.c_double*n)(*omega),
                            ctypes.c_int(spin), ctypes.c_int(deriv),
                            ctypes.c_int(nvar), ctypes.c_int(ngrids),
                            ctypes.c_int(outlen),
                            rho.ctypes.data_as(ctypes.c_void_p),
                            out.ctypes.data_as(ctypes.c_void_p),
                            ctypes.c_double(density_threshold))
    #out[1]vrho out[2]vsigma
    Exc= np.einsum('p,p,p->',out[0],rho[0],weights)
    print("vsigma",out[2].shape)
    Vxctmp = 0.5*np.einsum('pb,p,p,pa->ab', phi, out[1], weights, phi, optimize=True)
    Vxctmp += 2.0 *np.einsum('pb,p,p,p,pa->ab', phix, out[2], rho[1], weights, phi, optimize=True)
    Vxctmp += 2.0 *np.einsum('pb,p,p,p,pa->ab', phiy, out[2], rho[2], weights, phi, optimize=True)
    Vxctmp += 2.0 *np.einsum('pb,p,p,p,pa->ab', phiz, out[2], rho[3], weights, phi, optimize=True)

    #functional derivative 
    #sym
    Vxc=Vxctmp+Vxctmp.T
    #print(out.shape)
    return Exc, Vxc

In [9]:

# Exchange-correlation energy
print(f"Exchange-correlation energy: {mypbe.get_veff().exc} Hartree")
dm = mypbe.make_rdm1()
print(dm.shape)
rho_grid2=  numint.eval_rho(mol, ao_value_grid, dm,xctype='GGA')
print(rho_grid2.shape)
exctest,vxctest=evalexc(rho_grid2)
print(exctest)

Exchange-correlation energy: -0.5108681274032191 Hartree
(10, 10)
(4, 20048)
((101, 1), (130, 1))
vsigma (20048,)
-0.510868127515638


In [20]:
#define the following integrals using pyscf
T = mol.intor('int1e_kin') 
V = mol.intor('int1e_nuc') 
S = mol.intor('int1e_ovlp')
I = mol.intor('int2e')

#our core hamiltonian
H=T+V

#we need to construct the matrix that diagonalizes S
e, U = np.linalg.eigh(S)
A = U @ np.diag(e**-0.5) @ U.T

#We are doing a restricted calculation, so each orbital contains 2 electrons
ndocc= mol.nelectron // 2

# Initial guess at D using just H_core 

F_p = A.dot(H).dot(A)
# Diagonalize F_p for eigenvalues & eigenvectors with NumPy
e, C_p = np.linalg.eigh(F_p)
# Transform C_p back into AO basis
C = A.dot(C_p)
# Grab occupied orbitals
C_occ = C[:, :ndocc]
# Build density matrix from occupied orbitals
D = 2 * np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True)
print(D.shape)

(10, 10)


In [21]:
# Begin SCF Iterations
print('==> Starting SCF Iterations <==\n')
E_old = 0.0

#this is for DIIS
# Trial & Residual Vector Lists
F_list = []
DIIS_RESID = []


# Maximum SCF iterations
MAXITER = 40
# Energy convergence criterion
E_conv = 1.0e-8
D_conv = 1.0e-6

for scf_iter in range(1, MAXITER + 1):
    # Build Fock matrix
    #print(D)
    J = np.einsum('pqrs,rs->pq', I, D, optimize=True)
    rho_grid=  numint.eval_rho(mol, ao_value_grid, D,xctype='GGA')
    Exc , vxc = evalexc(rho_grid)
    F = H + J + vxc

    H_E = np.sum(D * H)
    J_E = 0.5 * np.sum(D * J)
        
    # Build DIIS Residual
    diis_r = A.dot(F.dot(D).dot(S) - S.dot(D).dot(F)).dot(A)
    # Append trial & residual vectors to lists
    F_list.append(F)
    DIIS_RESID.append(diis_r)
    
    
    SCF_E = H_E + J_E + Exc + mol.energy_nuc()#different 
    dE = SCF_E - E_old
    dRMS = np.mean(diis_r**2)**0.5
    print('SCF Iteration %3d: Energy = %4.16f dE = % 1.5E dRMS = %1.5E' % (scf_iter, SCF_E, dE, dRMS))

    # Check for convergence
    if (abs(dE) < E_conv) and (dRMS < D_conv):
        break
    E_old = SCF_E
    
    #DIIS procedure
    if scf_iter >= 2:
        # Build B matrix
        B_dim = len(F_list) + 1
        B = np.empty((B_dim, B_dim))
        B[-1, :] = -1
        B[:, -1] = -1
        B[-1, -1] = 0
        for i in range(len(F_list)):
            for j in range(len(F_list)):
                B[i, j] = np.einsum('ij,ij->', DIIS_RESID[i], DIIS_RESID[j], optimize=True)

        # Build RHS of Pulay equation 
        rhs = np.zeros((B_dim))
        rhs[-1] = -1
               
        # Solve Pulay equation for c_i's with NumPy
        coeff = np.linalg.solve(B, rhs)
        
        # Build DIIS Fock matrix
        F = np.zeros_like(F)
        for x in range(coeff.shape[0] - 1):
            F += coeff[x] * F_list[x]
    

    
    # Compute new orbital guess
    F_p = A.dot(F).dot(A)
    e, C_p = np.linalg.eigh(F_p)
    C = A.dot(C_p)
    C_occ = C[:, :ndocc]
    D = 2* np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True)

# Post iterations
print('\nSCF converged.')
print(f'Final RHF Energy: {SCF_E:.13f} [Eh]')
# get_veff calculates columb energy (J) + XC, and each part is accessbile with .ecoul or .exc
# Coulomb energy (Hartree energy)
print(f"Coulomb energy: {J_E} Hartree")
# Exchange-correlation energy
print(f"Exchange-correlation energy: {Exc} Hartree")

# Nuclear repulsion energy
print(f"Nuclear repulsion energy: {mol.energy_nuc()} Hartree")

-0.9956505424734725

==> Starting SCF Iterations <==

((101, 1), (130, 1))
vsigma (20048,)
SCF Iteration   1: Energy = -0.9872335472844047 dE = -9.87234E-01 dRMS = 1.97841E-02
((101, 1), (130, 1))
vsigma (20048,)
SCF Iteration   2: Energy = -0.9955853487420654 dE = -8.35180E-03 dRMS = 1.51157E-03
((101, 1), (130, 1))
vsigma (20048,)
SCF Iteration   3: Energy = -0.9956504556552908 dE = -6.51069E-05 dRMS = 5.28702E-05
((101, 1), (130, 1))
vsigma (20048,)
SCF Iteration   4: Energy = -0.9956505425756057 dE = -8.69203E-08 dRMS = 6.14134E-07
((101, 1), (130, 1))
vsigma (20048,)
SCF Iteration   5: Energy = -0.9956505425858937 dE = -1.02880E-11 dRMS = 2.47643E-10

SCF converged.
Final RHF Energy: -0.9956505425859 [Eh]
Coulomb energy: 0.8628345527725945 Hartree
Exchange-correlation energy: -0.5108681228758628 Hartree
Nuclear repulsion energy: 0.26458860546 Hartree


-0.9956505424734725

In [23]:
mol=gto.Mole()
mol.atom='''
    H 0 0 -1
    H 0 0 1
'''#Z-matrix input format
mol.basis='cc-pvdz'
mol.symmetry = True
mol.build()
mypbe = dft.RKS(mol)
mypbe.xc = 'pbe,pbe'
mypbe.kernel()

converged SCF energy = -0.995650542473473


-0.9956505424734725

In [22]:
# get_veff calculates columb energy (J) + XC, and each part is accessbile with .ecoul or .exc
# Coulomb energy (Hartree energy)
print(f"Coulomb energy: {mypbe.get_veff().ecoul} Hartree")
# Exchange-correlation energy
print(f"Exchange-correlation energy: {mypbe.get_veff().exc} Hartree")

# Nuclear repulsion energy
print(f"Nuclear repulsion energy: {mypbe.energy_nuc()} Hartree")

# note we can also access the electronic energy with energy_elec(), which returns: (electronic energy, 2electron contribution)
print(f"Electronic energy: {mypbe.energy_elec()[0]} Hartree")

Coulomb energy: 0.8628345586618538 Hartree
Exchange-correlation energy: -0.5108681274032194 Hartree
Nuclear repulsion energy: 0.26458860546 Hartree
Electronic energy: -1.2602391479334725 Hartree
